In [ ]:
#@title Install Z-Image-Turbo
#ComfyUI Installation
%cd /content/
!git clone https://github.com/comfyanonymous/ComfyUI

%cd /content/ComfyUI

PIN_COMMIT = "3c8456223c5f6a41af7d99219b391c8c58acb552"  #fixed version
# PIN_COMMIT="" #latest version
if PIN_COMMIT:
    !git fetch --all -q
    !git reset --hard {PIN_COMMIT}


!wget https://raw.githubusercontent.com/NeuralFalconYT/Z-Image-Colab/refs/heads/main/app.py

!pip install -r requirements.txt

#Model Download

!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/T5B/Z-Image-Turbo-FP8/resolve/main/z-image-turbo-fp8-e4m3fn.safetensors -d /content/ComfyUI/models/diffusion_models -o z-image-turbo-fp8-e4m3fn.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors -d /content/ComfyUI/models/clip -o qwen_3_4b.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/vae/ae.safetensors -d /content/ComfyUI/models/vae -o ae.safetensors

#Clear Terminal
from IPython.display import clear_output
clear_output()
print("\033[92mZ-Image-Turbo Installation successful\033[0m")
import os

paths = [
    "/content/ComfyUI/models/diffusion_models/z-image-turbo-fp8-e4m3fn.safetensors",
    "/content/ComfyUI/models/clip/qwen_3_4b.safetensors",
    "/content/ComfyUI/models/vae/ae.safetensors",
]

for p in paths:
    if not os.path.exists(p):
        print(f"\033[91mMISSING (possibly HuggingFace blocked z-image-turbo model download, Run the Colab Cell again): {p}\033[0m")

#@title Keep Alive for Mobile Users
from IPython.display import Audio,display
display(Audio("https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a", autoplay=True))

# Run from colab cell

In [ ]:
#@title Utils Code
%cd /content/ComfyUI

import os, random, time

import torch
import numpy as np
from PIL import Image
import re, uuid
from nodes import NODE_CLASS_MAPPINGS

UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
CLIPLoader = NODE_CLASS_MAPPINGS["CLIPLoader"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
KSampler = NODE_CLASS_MAPPINGS["KSampler"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    unet = UNETLoader.load_unet("z-image-turbo-fp8-e4m3fn.safetensors", "fp8_e4m3fn_fast")[0]
    clip = CLIPLoader.load_clip("qwen_3_4b.safetensors", type="lumina2")[0]
    vae = VAELoader.load_vae("ae.safetensors")[0]

save_dir="./results"
os.makedirs(save_dir, exist_ok=True)
def get_save_path(prompt):
  save_dir = "./results"
  safe_prompt = re.sub(r'[^a-zA-Z0-9_-]', '_', prompt)[:25]
  uid = uuid.uuid4().hex[:6]
  filename = f"{safe_prompt}_{uid}.png"
  path = os.path.join(save_dir, filename)
  return path

@torch.inference_mode()
def generate(input):
    values = input["input"]
    positive_prompt = values['positive_prompt']
    negative_prompt = values['negative_prompt']
    seed = values['seed'] # 0
    steps = values['steps'] # 9
    cfg = values['cfg'] # 1.0
    sampler_name = values['sampler_name'] # euler
    scheduler = values['scheduler'] # simple
    denoise = values['denoise'] # 1.0
    width = values['width'] # 1024
    height = values['height'] # 1024
    batch_size = values['batch_size'] # 1.0

    if seed == 0:
        random.seed(int(time.time()))
        seed = random.randint(0, 18446744073709551615)

    positive = CLIPTextEncode.encode(clip, positive_prompt)[0]
    negative = CLIPTextEncode.encode(clip, negative_prompt)[0]
    latent_image = EmptyLatentImage.generate(width, height, batch_size=batch_size)[0]
    samples = KSampler.sample(unet, seed, steps, cfg, sampler_name, scheduler, positive, negative, latent_image, denoise=denoise)[0]
    decoded = VAEDecode.decode(vae, samples)[0].detach()
    save_path=get_save_path(positive_prompt)
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(save_path)
    return save_path,seed

In [ ]:
# ==========================================
# 🔥 AUTO START API (Run All & Relax)
# ==========================================
import os, time, sys, threading, subprocess

# ১. অটো সেটআপ
try:
    import uvicorn, nest_asyncio, pyngrok
    from fastapi import FastAPI, Response
    from pydantic import BaseModel
except:
    os.system("pip install -q fastapi uvicorn pyngrok nest-asyncio")
    import uvicorn, nest_asyncio, pyngrok
    from fastapi import FastAPI, Response
    from pydantic import BaseModel

os.system("fuser -k 8000/tcp")

# ২. অ্যাপ কনফিগারেশন
app = FastAPI()
queue_lock = threading.Lock()

class UserRequest(BaseModel):
    positive_prompt: str
    negative_prompt: str = "low quality, bad anatomy, blurry"
    width: int = 720
    height: int = 1280
    steps: int = 10
    cfg: float = 1.0
    seed: int = 0

@app.post("/generate")
def api_generate(req: UserRequest):
    with queue_lock:
        input_data = {
          "input": {
            "positive_prompt": req.positive_prompt,
            "negative_prompt": req.negative_prompt,
            "width": req.width, "height": req.height,
            "batch_size": 1, "seed": req.seed, "steps": req.steps,
            "cfg": req.cfg, "sampler_name": "euler", "scheduler": "simple", "denoise": 1.0,
          }
        }
        try:
            if 'generate' not in globals(): return Response(content="Model Loading... Please wait.", status_code=500)
            image_path, seed = generate(input_data)
            with open(image_path, "rb") as f: img_bytes = f.read()
            return Response(content=img_bytes, media_type="image/png")
        except Exception as e: return Response(content=str(e), status_code=500)

def keep_alive():
    while True: time.sleep(60); print(".", end="", flush=True)
threading.Thread(target=keep_alive, daemon=True).start()

# ৩. টোকেন সেটআপ (এখানে আপনার টোকেন দিন)
ngrok_token = "39MLZulT6VaVF91NJwfCkdTJlkl_7yQUmuyN8bWxPHXYVkU2r"
ngrok.set_auth_token(ngrok_token)

try:
    public_url = ngrok.connect(8000).public_url
    print(f"\n🚀 API LINK: {public_url}/generate\n")
except: pass

nest_asyncio.apply()
uvicorn.run(app, port=8000)